In [1]:
# Convolutional network with Tensorflow
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

sess = tf.InteractiveSession()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
# placeholders
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [17]:
# 2 functions for bias and weights variables
# initialization with a small values in order to prevent symmetry and dead neurons
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [18]:
# 2 functions for convolution and pooling
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [19]:
# first convolutional layer
W_conv1 = weight_variable([5, 5, 1, 32])  # 5*5 is patch size and 1 is the input dimension and 32 is the output dimension
b_conv1 = bias_variable([32])             # since we have 32 neurons

In [20]:
x_image = tf.reshape(x, [-1, 28, 28, 1])  # 28*28 is the size of picture and 1 is the number of color channels

In [21]:
# compute convolutional and max pooling for the first layer
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [22]:
# second convolutional layer
W_conv2 = weight_variable([5, 5, 32, 64])  # 32 input and 64 output
b_conv2 = bias_variable([64])              # since we have 64 neurons

In [23]:
# compute convolutional and max pooling for the second layer
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [24]:
# densely fully connected layer
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [25]:
# dropout: we send a placeholder for that in order to enable it in training and disable it in testing
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [26]:
# softmax layer to compute the distribution for the final layer
W_fc2 = weight_variable([1024, 10])    # 10 because we have 10 classes (of digits)
b_fc2 = bias_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [27]:
# train and evaluate the model
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())
for i in range(500):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.1
step 100, training accuracy 0.78
step 200, training accuracy 0.92
step 300, training accuracy 0.88
step 400, training accuracy 0.98
test accuracy 0.9436


In [28]:
W_conv2.eval()

array([[[[  6.06222376e-02,  -4.15037461e-02,  -9.46073383e-02, ...,
           -1.27432287e-01,   1.24885151e-02,  -4.48169606e-03],
         [ -1.73523335e-03,   1.66072443e-01,   2.30698809e-02, ...,
           -6.50816709e-02,   5.83048314e-02,  -3.69798727e-02],
         [ -1.41949832e-01,  -7.57733872e-03,  -5.51790185e-02, ...,
            1.77838460e-01,  -4.55179401e-02,   3.90326902e-02],
         ..., 
         [  4.85178865e-02,  -3.02337557e-02,   2.64621507e-02, ...,
           -5.42729534e-02,  -1.85955912e-01,   5.49234301e-02],
         [ -1.51077285e-01,  -7.19259456e-02,   4.56884392e-02, ...,
           -2.49909125e-02,   4.95387502e-02,   2.99504623e-02],
         [  4.63772826e-02,   4.85757403e-02,  -7.28262961e-02, ...,
           -4.10926342e-02,  -2.77455393e-02,  -1.26076221e-01]],

        [[  3.29840928e-02,   1.80236995e-01,   1.30476296e-01, ...,
           -4.44657803e-02,   4.57179733e-02,  -6.10540435e-02],
         [ -8.15590546e-02,   6.83070067e-03,